# Objective

Currently, each row has all the stats needed for the team in the 'team' column. However, we don't have the same information for the opponent in the same row. Let's make that happen.

To further clarify, for each row, we need to pull the opposing team's rolling stats, adding them back into the same row. This way, our model will only need one row to make a prediction.

In [1]:
import pandas as pd
import numpy as np
from clean import clean_games

In [2]:
pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 73)

In [3]:
game_df = clean_games("data/games_scraped.pickle")

In [4]:
mask = (game_df.game_id == 'chi-rai-2019-10-06')
subset = game_df.loc[mask]
subset

,team,year,team_year,date,opp,week_num,decade,game_day_of_week,game_outcome,team_record,pts_off,margin,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,game_time,overtime,exp_pts_off,exp_pts_def,exp_pts_st,home,game_id,result_tie,result_win,wins,losses,ties,prev_wins,prev_losses,prev_ties,roll3_ties,roll3_wins,prev_pts_off,prev_pts_def,prev_margin,prev_first_down_off,prev_yards_off,prev_pass_yds_off,prev_rush_yds_off,prev_to_off,prev_first_down_def,prev_yards_def,prev_pass_yds_def,prev_rush_yds_def,prev_to_def,prev_result_tie,prev_result_win,roll3_pts_off,roll3_pts_def,roll3_margin,roll3_first_down_off,roll3_yards_off,roll3_pass_yds_off,roll3_rush_yds_off,roll3_to_off,roll3_yards_def,roll3_pass_yds_def,roll3_rush_yds_def,roll3_to_def,ewma_pts_off,ewma_pts_def,ewma_margin,ewma_first_down_off,ewma_yards_off,ewma_pass_yds_off,ewma_rush_yds_off,ewma_to_off,ewma_yards_def,ewma_pass_yds_def,ewma_rush_yds_def,ewma_to_def
4823,chi,2019,chi-2019,2019-10-06,rai,5,201,Sun,L,3-2,21.0,-3.0,24.0,15.0,236.0,194.0,42.0,2.0,25.0,398.0,229.0,169.0,2.0,1.0,NaN,-3.0,-2.0,-0.0,0,chi-rai-2019-10-06,0,0,3,2,0,3,1,0,3.0,3.0,16.0,6.0,10.0,17.0,269.0,197.0,72.0,0.0,15.0,222.0,182.0,40.0,2.0,0.0,1.0,21.000,11.667,9.333,18.667,280.000,175.000,105.0,0.333,316.667,250.333,66.333,2.667,17.286,11.053,6.233,18.082,274.496,184.250,90.246,0.469,290.353,229.646,60.707,2.148
20606,rai,2019,rai-2019,2019-10-06,chi,5,201,Sun,W,3-2,24.0,3.0,21.0,25.0,398.0,229.0,169.0,2.0,15.0,236.0,194.0,42.0,2.0,1.0,NaN,2.0,3.0,0.0,1,chi-rai-2019-10-06,0,1,3,2,0,2,2,0,1.0,1.0,31.0,24.0,7.0,21.0,377.0,189.0,188.0,1.0,22.0,346.0,265.0,81.0,2.0,0.0,1.0,18.333,28.667,-10.333,19.000,328.667,193.667,135.0,1.333,399.333,291.667,107.667,1.000,20.200,25.924,-5.725,19.070,336.851,207.394,129.456,1.027,384.007,277.499,106.508,0.830


So, in the above table, I need to get all of the `roll3_x` and `ewma_x` columns from the adjacent tables, respectively. We'll prefix these with ``opp_``.

In [5]:
game_df.columns

Index(['team', 'year', 'team_year', 'date', 'opp', 'week_num', 'decade',
       'game_day_of_week', 'game_outcome', 'team_record', 'pts_off', 'margin',
       'pts_def', 'first_down_off', 'yards_off', 'pass_yds_off',
       'rush_yds_off', 'to_off', 'first_down_def', 'yards_def', 'pass_yds_def',
       'rush_yds_def', 'to_def', 'game_time', 'overtime', 'exp_pts_off',
       'exp_pts_def', 'exp_pts_st', 'home', 'game_id', 'result_tie',
       'result_win', 'wins', 'losses', 'ties', 'prev_wins', 'prev_losses',
       'prev_ties', 'roll3_ties', 'roll3_wins', 'prev_pts_off', 'prev_pts_def',
       'prev_margin', 'prev_first_down_off', 'prev_yards_off',
       'prev_pass_yds_off', 'prev_rush_yds_off', 'prev_to_off',
       'prev_first_down_def', 'prev_yards_def', 'prev_pass_yds_def',
       'prev_rush_yds_def', 'prev_to_def', 'prev_result_tie',
       'prev_result_win', 'roll3_pts_off', 'roll3_pts_def', 'roll3_margin',
       'roll3_first_down_off', 'roll3_yards_off', 'roll3_pass_yds_off',


In [6]:
opp_pull_cols = ['game_id', 'team', 'opp', 'prev_wins', 'prev_losses',
       'prev_ties', 'roll3_ties', 'roll3_wins', 'prev_pts_off', 'prev_pts_def',
       'prev_margin', 'prev_first_down_off', 'prev_yards_off',
       'prev_pass_yds_off', 'prev_rush_yds_off', 'prev_to_off',
       'prev_first_down_def', 'prev_yards_def', 'prev_pass_yds_def',
       'prev_rush_yds_def', 'prev_to_def', 'prev_result_tie',
       'prev_result_win', 'roll3_pts_off', 'roll3_pts_def', 'roll3_margin',
       'roll3_first_down_off', 'roll3_yards_off', 'roll3_pass_yds_off',
       'roll3_rush_yds_off', 'roll3_to_off', 'roll3_yards_def',
       'roll3_pass_yds_def', 'roll3_rush_yds_def', 'roll3_to_def',
       'ewma_pts_off', 'ewma_pts_def', 'ewma_margin', 'ewma_first_down_off',
       'ewma_yards_off', 'ewma_pass_yds_off', 'ewma_rush_yds_off',
       'ewma_to_off', 'ewma_yards_def', 'ewma_pass_yds_def',
       'ewma_rush_yds_def', 'ewma_to_def']

In [7]:
# convert all cols to float
for col in opp_pull_cols[3:]:
    game_df[col] = game_df[col].astype(float)

In [44]:
mask = game_df.team_year == 'chi-2019'
game_df[mask][opp_pull_cols].head(2)

,game_id,team,opp,prev_wins,prev_losses,prev_ties,roll3_ties,roll3_wins,prev_pts_off,prev_pts_def,prev_margin,prev_first_down_off,prev_yards_off,prev_pass_yds_off,prev_rush_yds_off,prev_to_off,prev_first_down_def,prev_yards_def,prev_pass_yds_def,prev_rush_yds_def,prev_to_def,prev_result_tie,prev_result_win,roll3_pts_off,roll3_pts_def,roll3_margin,roll3_first_down_off,roll3_yards_off,roll3_pass_yds_off,roll3_rush_yds_off,roll3_to_off,roll3_yards_def,roll3_pass_yds_def,roll3_rush_yds_def,roll3_to_def,ewma_pts_off,ewma_pts_def,ewma_margin,ewma_first_down_off,ewma_yards_off,ewma_pass_yds_off,ewma_rush_yds_off,ewma_to_off,ewma_yards_def,ewma_pass_yds_def,ewma_rush_yds_def,ewma_to_def
4822,chi-min-2019-09-29,chi,min,2.0,1.0,0.0,2.0,2.0,31.0,15.0,16.0,21.0,298.0,208.0,90.0,1.0,25.0,356.0,287.0,69.0,5.0,0.0,1.0,16.667,13.000,3.667,18.333,275.0,178.667,96.333,0.667,313.667,245.000,68.667,2.000,17.834,13.205,4.628,18.542,276.836,178.819,98.017,0.668,319.466,249.939,69.527,2.211
4823,chi-rai-2019-10-06,chi,rai,3.0,1.0,0.0,3.0,3.0,16.0,6.0,10.0,17.0,269.0,197.0,72.0,0.0,15.0,222.0,182.0,40.0,2.0,0.0,1.0,21.000,11.667,9.333,18.667,280.0,175.000,105.000,0.333,316.667,250.333,66.333,2.667,17.286,11.053,6.233,18.082,274.496,184.250,90.246,0.469,290.353,229.646,60.707,2.148


In [9]:
merged_df = (game_df
             .merge(right=game_df[opp_pull_cols],
                    left_on=['game_id', 'team'],
                    right_on=['game_id', 'opp'],
                    suffixes=[None, '_opp']))

In [41]:
mask = (merged_df.game_id == 'dal-nyg-2019-11-04')

Let's check another one.

In [42]:
merged_df[mask][['game_id', 'team', 'opp', 'prev_rush_yds_off',
                 'prev_rush_yds_off_opp', 'roll3_margin', 'roll3_margin_opp']]

,game_id,team,opp,prev_rush_yds_off,prev_rush_yds_off_opp,roll3_margin,roll3_margin_opp
2946,chi-gnb-1960-12-04,chi,gnb,221.0,118.0,9.000,6.333
7479,chi-gnb-1960-12-04,gnb,chi,118.0,221.0,6.333,9.000


And now a much older one.

In [43]:
mask = (merged_df.game_id == 'chi-gnb-1960-12-04')
merged_df[mask][['game_id', 'team', 'opp', 'prev_rush_yds_off',
                 'prev_rush_yds_off_opp', 'roll3_margin', 'roll3_margin_opp']]

,game_id,team,opp,prev_rush_yds_off,prev_rush_yds_off_opp,roll3_margin,roll3_margin_opp
2946,chi-gnb-1960-12-04,chi,gnb,221.0,118.0,9.000,6.333
7479,chi-gnb-1960-12-04,gnb,chi,118.0,221.0,6.333,9.000


In [45]:
game_df.shape

(22577, 79)